In [ ]:
import numpy as np
from sklearn.preprocessing import normalize
import re
import sklearn
from sklearn import decomposition
import seaborn
from math import log

In [ ]:
def pagerank(A, eps=0.0001, d=0.85, max_iter = 5000):
    """
    PageRank algorithm.
    
    Given a similarity matrix, returns list of score for each sentence.

    :param A:         matrix (n, n)
                      The adjacency matrix of the graph on which to compute PageRank score
                      
    :param eps:       float, optional
                      Tolerance. The algorithm stops as soon as the update magnitude of all
                      values is below this threshold.
              
    :param d:         float, optional
                      1 - probability of teleporting to a random node
            
    :param max_iter:  int, optional
                      Maximum number of iterations
                      
    :return:    A matrix (n, n).
                The ranking of sentences in the document.
    """
    
    # P is the vector of probability to "teleport" on each node. By default filled of 1/n.
    P = np.ones(len(A)) / len(A)
    
    while max_iter > 0 :
        max_iter-=1
        # Markov chain transition
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.dot(P)
        # Normalization
        new_P = new_P / np.linalg.norm(new_P)
        # Compute mean absolute error (MAE)
        delta = abs(new_P - P).sum() / len(new_P)
        if delta <= eps:
            return new_P
        P = new_P
    
    print("Convergence error : " + str(delta))
    return new_P

In [ ]:
def tr_summarizer(matrix, corpus, weights=None, nb_words = 100, diag = "none", bias = None):
    """
    Prepare the sentence-term matrix before applying PageTank algorithm to it.
    
    :param matrix:    matrix (n, m)
                      A sentence-term matrix weighting the importance of a term for a sentence.
                      
    :param corpus:    list of string.
                      A single document.
    
    :param weights:   ???
    :param nb_words:  The number of words for the summary.
    :param diag:      ???
    :param biais:     ???
    """
    matrix = np.array(matrix)
    
    if diag == "before":
         np.fill_diagonal(matrix,0)   
    
    # Similarity matrix <=> Adjacancy matrix ???
    sim_matrix = normalize(matrix , norm = 'l1', axis = 0)
    
    if (not weights is None) and (len(weights) == matrix.shape[0]):
        sim_matrix = np.matmul(sim_matrix,np.diag(weights))
    
    if diag == "after" :
        np.fill_diagonal(matrix,0)

    if bias is not None:
        sim_matrix = sim_matrix + bias
    sim_matrix = sim_matrix / matrix.shape[0]
    results = pagerank(sim_matrix)

    return results

# LSA : Latent Semantic Analysis

Source: [Text Summarization Techniques: A Brief Survey](https://arxiv.org/pdf/1707.02268.pdf)

## 1. Builds term-sentence matrix

$X \in \mathbb{M}^{n \times m}$: term sentence matrix
* rows correspond to a word from the input (n words)
* columns correspond to a sentence (m sentences)

$x_{ij}$: weight of the word j in sentence i
* weights of the words computed with TFIDF.
* if a sentence does not have a word the weight of that word in the sentence is zero.

## 2. Sentence correlation matrix over the vocabulary

$XX^T$

## 3. SVD: singular value decomposition

Transforms the matrix X into three matrices: $A = U\Sigma V^T$.

$XX^T = (U\Sigma V^T)(U\Sigma V^T)^T = (U\Sigma V^T) (V \Sigma^T U^T) = U\Sigma \Sigma^T U^T $

In [ ]:
def lsa_summarizer(matrix, corpus, weights=None, nbcompfun = None, nb_words = 100, diag = "none", bias = None):
    """
    :param matrix:    matrix (n, m)
                      sentence-term matrix weighting the importance of a term for a sentence.
    
    :param corpus:    list of string.
                      A single document.
    :param weights:   ???
    :param nbcompfun: ???
    :param nb_words:  The number of words for the summary.
    :param diag:      ???
    :param biais:     ???
    
    """
    #Méthode de calcul LSA
    matrix = np.array(matrix)
    
    if nbcompfun == None:
        nbcompfun = lambda x : log(x)
    
    k = max(1, int(nbcompfun(len(corpus))))
    
    if k >= len(corpus):
        return np.sum(matrix,axis=1)
    
    if diag == "before" :
        np.fill_diagonal(matrix,0) 

    sim_matrix = normalize(matrix , norm = 'l1', axis = 0)
    
    if (not weights is None) and (len(weights) == matrix.shape[0]):
        sim_matrix = np.matmul(sim_matrix,np.diag(weights))
    
    if diag == "after" :
        np.fill_diagonal(matrix,0)
    if bias is not None:
        sim_matrix = sim_matrix + bias
    sim_matrix = sim_matrix / matrix.shape[0]

    #tsvd = sklearn.decomposition.TruncatedSVD(k, random_state=1337)
    tsvd = sklearn.decomposition.TruncatedSVD(k, random_state=1337, algorithm = "arpack")
    
    # Dimension reduction of sim_matrix using truncated SVD.
    # results = topic-sentence matrix
    results = tsvd.fit_transform(sim_matrix)
    
    # scores = sigular_values @ result.T
    # scores matrix describe how much a sentence represent a word, thus
    # the weight / importance of a word in a sentence.
    scores = np.abs(np.matmul(results, np.diag(np.sqrt(tsvd.singular_values_))))
    
    # Give a score per sentence as the sum of the weights of the words it contains
    # score_sent = sum(row)
    # REMARK : Text Summarization Techniques: A Brief survey (paper) suggest sqrt(sum(scores_ij^2))
    maxscores = np.sum(scores,axis=1)

    return maxscores

In [ ]:
def generic_summarizer(method, matrix, corpus, weighted, lsanbcompfun = lambda x : log(x), diag = "none", bias = None):
    """
    :param method:        String defining if summarization is done with LSA or TextRank.
    :param matrix:        Adjancy matrix. Distance between sentences.
    :param corpus:        Array of string. One textual document.
    :param weighted:      Boolean.
    :param lsandcompfun:
    :param diag:
    :param bias:    Dictionay mapping element of interests (ex html tag) to value, weight.
    """
    
    segments, bias_dict = bias
    if bias_dict is not None:
        doc_bias = get_bias(corpus, segments, bias_dict)
    else:
        doc_bias = None
    #Is used to determine which method has to be used
    if not lsanbcompfun : 
        lsanbcompfun = lambda x : 1
    if method == "tr":
            if weighted:
                return tr_summarizer(matrix, corpus, get_weights(corpus), diag = diag, bias = doc_bias)
            else:
                return tr_summarizer(matrix, corpus, diag = diag, bias = doc_bias)
    else:
        if method == "lsa":
            if weighted:
                return lsa_summarizer(matrix, corpus, get_weights(corpus),
                                      nbcompfun = lsanbcompfun, diag = diag , bias = doc_bias)
            else:
                return lsa_summarizer(matrix, corpus, nbcompfun = lsanbcompfun, diag = diag, bias = doc_bias)

## Document weighting

Defines bias matrix for the document, given a bias dictionary and the segments of interest of a document.

In [ ]:
def get_bias(doc, segments, bias_dict):
    """
    Defines the bias matrix for the document.
    bias param is not the matrix. It's named bias only to respect the old format defined by prev devs.

    Given a dictionary mapping sentences token to tag, builds a vector with
    ith sentence weight at pos i.
    
    :param corpus:    one preprocessed document, as given from the pool shit bellow.
    :param segments:  Given an element (for instance, and html tag), maps the sentences of the
                      document which were concerned by the document.
    :param bias:      Given an element (for instance, an html tag), return a weight value.
    
    
    """
    w = np.zeros(len(doc))
    for tag, sents in segments.items():
        # get sentences position in document
        for sent in sents:
            if len(sent) < 1 or sent not in doc:
                continue
            sent_id = doc.index(sent)
            w[sent_id] = bias_dict[tag]
    return w

Step 0

We define $W \in \mathbf{R}^n$ the intermediate weight vector:

$w_{i} = \large \frac{len(s_i)}{\sqrt{n + 1 - i}} $

* $d \in \mathbb{R}^n$: document
* $s_i$: ith sentence of document d, $i \in [[0, n]]$
* $w_{i}$: weight for ith sentence


Step 1
We define $\Omega \in \mathbf{R}^n$ the final weight vector:

$\Omega_i = n \times \large \frac{w_{i}}{\sum_{i = 1}^{n} w_i}$

In [ ]:
def get_weights(corpus):
    """
    Build a weight matrix for a document.
    Attributes to each sentence a weight.
    Weights are defined <???>
    """
    weights = []
    nbsent = len(corpus)
    sentindex = nbsent + 1
    for sent in corpus :
        splitsent = sent.split()
        weights.append(len(splitsent) / ((sentindex**0.5)))
        sentindex -= 1.0        
    weights = np.array(weights)
    weights = nbsent * weights / np.sum(weights)
    return weights  